# 🎓 Lecture 18: Advanced Unsupervised Learning
## Hands-on Practice Notebook

---

**Based on**: Ho-min Park's Lecture

**Topics Covered**:
1. 🔄 Self-Supervised Learning (SimCLR, MoCo, BYOL)
2. 📈 Time Series Clustering (DTW, K-Shape)
3. 🕸️ Graph Clustering (Spectral, Louvain, GNN)
4. 🚀 Advanced Topics (Multi-modal, Large-scale)

**Estimated Time**: 4-6 hours

**Prerequisites**:
- Python programming
- Basic machine learning concepts
- NumPy, Pandas, Matplotlib
- PyTorch basics (optional but recommended)

---

### 📚 Learning Objectives

By the end of this notebook, you will be able to:
- Understand and implement contrastive learning principles
- Apply Dynamic Time Warping for time series clustering
- Perform graph clustering using spectral methods and GNNs
- Handle multi-modal and large-scale clustering problems
- Compare different clustering approaches on real datasets

---

## 📦 Part 1: Setup and Imports

Let's import all necessary libraries for our hands-on exercises.

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.spatial.distance import euclidean, cdist
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
import warnings
warnings.filterwarnings('ignore')

# Sklearn utilities
from sklearn.cluster import KMeans, SpectralClustering, DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score, adjusted_rand_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.datasets import make_blobs, make_moons, make_circles

# Deep Learning
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms

# Graph libraries
import networkx as nx

# Plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

print("✅ All libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"NetworkX version: {nx.__version__}")

---

# 🔄 Part 2: Self-Supervised Learning

Self-supervised learning enables models to learn representations from unlabeled data by creating pseudo-labels from the data itself.

## Key Concepts:
- **Contrastive Learning**: Learn by contrasting positive pairs (similar) against negative pairs (dissimilar)
- **SimCLR**: Simple Framework for Contrastive Learning of Visual Representations
- **InfoNCE Loss**: Normalized Temperature-scaled Cross Entropy Loss

---

## 📝 Exercise 1: Understanding Contrastive Learning

### Concept

**Contrastive learning** aims to learn representations where:
- Similar items (positive pairs) are pulled **close together**
- Dissimilar items (negative pairs) are pushed **far apart**

**InfoNCE Loss Formula**:

$$\mathcal{L} = -\log \frac{\exp(\text{sim}(z_i, z_j) / \tau)}{\sum_{k=1}^{2N} \mathbb{1}_{[k \neq i]} \exp(\text{sim}(z_i, z_k) / \tau)}$$

Where:
- $z_i, z_j$ are embeddings of positive pairs
- $\tau$ is the temperature parameter
- $\text{sim}$ is cosine similarity

### Implementation

Let's implement the InfoNCE loss function:

In [ ]:
def contrastive_loss(z_i, z_j, temperature=0.5):
    """
    Compute InfoNCE (NT-Xent) contrastive loss
    
    Args:
        z_i: embeddings of augmented view 1, shape (batch_size, embedding_dim)
        z_j: embeddings of augmented view 2, shape (batch_size, embedding_dim)
        temperature: temperature scaling parameter
    
    Returns:
        loss: contrastive loss value
    """
    batch_size = z_i.shape[0]
    
    # Normalize embeddings
    z_i = F.normalize(z_i, dim=1)
    z_j = F.normalize(z_j, dim=1)
    
    # Concatenate embeddings
    representations = torch.cat([z_i, z_j], dim=0)
    
    # Compute similarity matrix
    similarity_matrix = F.cosine_similarity(
        representations.unsqueeze(1), 
        representations.unsqueeze(0), 
        dim=2
    )
    
    # Create mask to remove self-similarities
    mask = torch.eye(2 * batch_size, dtype=torch.bool)
    similarity_matrix = similarity_matrix[~mask].view(2 * batch_size, -1)
    
    # Positive pairs are (i, j) and (j, i)
    positives = torch.cat([z_i @ z_j.T, z_j @ z_i.T], dim=0).diag()
    
    # Compute logits
    logits = similarity_matrix / temperature
    positives = positives / temperature
    
    # InfoNCE loss
    loss = -positives + torch.logsumexp(logits, dim=1)
    return loss.mean()


# Generate synthetic embeddings for demonstration
batch_size = 128
embedding_dim = 64

# Create positive pairs (similar embeddings with small noise)
z_anchor = torch.randn(batch_size, embedding_dim)
z_positive = z_anchor + torch.randn(batch_size, embedding_dim) * 0.1

# Compute loss
loss = contrastive_loss(z_anchor, z_positive, temperature=0.5)
print(f"✅ Contrastive Loss: {loss.item():.4f}")

# Test with different temperatures
temperatures = [0.1, 0.5, 1.0, 2.0]
losses = []

for temp in temperatures:
    loss = contrastive_loss(z_anchor, z_positive, temperature=temp)
    losses.append(loss.item())
    print(f"Temperature {temp:.1f}: Loss = {loss.item():.4f}")

### 📊 Visualization: Effect of Temperature

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Temperature vs Loss
axes[0].plot(temperatures, losses, 'o-', linewidth=2, markersize=8)
axes[0].set_xlabel('Temperature (τ)', fontsize=12)
axes[0].set_ylabel('Contrastive Loss', fontsize=12)
axes[0].set_title('Effect of Temperature on Contrastive Loss', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Plot 2: Embedding space visualization
# Generate 2D embeddings for visualization
n_samples = 50
z_2d_anchor = torch.randn(n_samples, 2)
z_2d_positive = z_2d_anchor + torch.randn(n_samples, 2) * 0.2
z_2d_negative = torch.randn(n_samples, 2) * 2

axes[1].scatter(z_2d_anchor[:, 0], z_2d_anchor[:, 1], 
                c='blue', alpha=0.6, s=100, label='Anchor', edgecolors='black')
axes[1].scatter(z_2d_positive[:, 0], z_2d_positive[:, 1], 
                c='green', alpha=0.6, s=100, label='Positive', edgecolors='black')
axes[1].scatter(z_2d_negative[:, 0], z_2d_negative[:, 1], 
                c='red', alpha=0.6, s=100, label='Negative', edgecolors='black')

# Draw lines connecting positive pairs
for i in range(min(20, n_samples)):
    axes[1].plot([z_2d_anchor[i, 0], z_2d_positive[i, 0]], 
                 [z_2d_anchor[i, 1], z_2d_positive[i, 1]], 
                 'gray', alpha=0.3, linewidth=1)

axes[1].set_xlabel('Dimension 1', fontsize=12)
axes[1].set_ylabel('Dimension 2', fontsize=12)
axes[1].set_title('Embedding Space: Positive vs Negative Pairs', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Key Insights:")
print("1. Lower temperature → More emphasis on hard negatives")
print("2. Higher temperature → Softer distributions")
print("3. Positive pairs should be close, negatives far apart")

### 🎯 Your Turn: Practice Task

**Task**: Modify the contrastive loss function to:
1. Add a margin-based loss component
2. Experiment with different similarity metrics (L2 distance instead of cosine)
3. Visualize how the embedding space changes with different parameters

**Hint**: Try implementing Triplet Loss as an alternative to InfoNCE.

---

## 📝 Exercise 2: SimCLR Data Augmentation Pipeline

### Concept

**SimCLR** (Simple Framework for Contrastive Learning) uses strong data augmentation to create positive pairs:

**Augmentation Pipeline**:
1. Random crop and resize
2. Random color jittering
3. Random Gaussian blur
4. Random horizontal flip

The key insight: Different augmented views of the same image should have similar representations.

### Implementation

In [ ]:
class SimCLRAugmentation:
    """
    SimCLR data augmentation pipeline
    """
    def __init__(self, img_size=32):
        self.transform = transforms.Compose([
            transforms.RandomResizedCrop(img_size, scale=(0.2, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomApply([
                transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)
            ], p=0.8),
            transforms.RandomGrayscale(p=0.2),
            transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.5),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])
    
    def __call__(self, x):
        return self.transform(x), self.transform(x)


# Load CIFAR-10 dataset (we'll use a small subset for demonstration)
from torchvision.datasets import CIFAR10
from PIL import Image

# Download and prepare dataset
try:
    dataset = CIFAR10(root='./data', train=True, download=True)
    print("✅ CIFAR-10 dataset loaded successfully!")
except:
    print("⚠️  CIFAR-10 download may take a moment...")
    dataset = CIFAR10(root='./data', train=True, download=True)

# Get a sample image
sample_img, label = dataset[42]
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
               'dog', 'frog', 'horse', 'ship', 'truck']

print(f"Sample image: {class_names[label]}")

# Apply SimCLR augmentations
augmenter = SimCLRAugmentation(img_size=32)

# Create augmented views
view1_list = []
view2_list = []

for _ in range(4):
    view1, view2 = augmenter(sample_img)
    view1_list.append(view1)
    view2_list.append(view2)

print(f"✅ Generated {len(view1_list)} augmented pairs")

### 📊 Visualization: Augmented Pairs

In [ ]:
def denormalize(tensor):
    """Denormalize tensor for visualization"""
    tensor = tensor * 0.5 + 0.5
    return torch.clamp(tensor, 0, 1)

fig, axes = plt.subplots(4, 3, figsize=(12, 14))

# Original image
for i in range(4):
    axes[i, 0].imshow(sample_img)
    axes[i, 0].set_title('Original', fontsize=12, fontweight='bold')
    axes[i, 0].axis('off')

# Augmented views
for i in range(4):
    # View 1
    img1 = denormalize(view1_list[i]).permute(1, 2, 0).numpy()
    axes[i, 1].imshow(img1)
    axes[i, 1].set_title('Augmented View 1', fontsize=12, fontweight='bold')
    axes[i, 1].axis('off')
    
    # View 2
    img2 = denormalize(view2_list[i]).permute(1, 2, 0).numpy()
    axes[i, 2].imshow(img2)
    axes[i, 2].set_title('Augmented View 2', fontsize=12, fontweight='bold')
    axes[i, 2].axis('off')

plt.suptitle(f'SimCLR Augmentation: {class_names[label].upper()}', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print("\n📊 Key Insights:")
print("1. Strong augmentations create diverse views")
print("2. Each pair should maintain semantic content")
print("3. Model learns to recognize invariances")

### 🎯 Your Turn: Practice Task

**Task**: Create a custom augmentation pipeline that:
1. Adds rotation augmentation (0°, 90°, 180°, 270°)
2. Implements cutout/random erasing
3. Compares augmentation strength effects on downstream performance

**Bonus**: Visualize which augmentations are most beneficial for learning.

---

## 📝 Exercise 3: Training a Simple Contrastive Encoder

### Concept

**Self-Supervised Pretraining Pipeline**:
1. **Pretrain**: Learn representations using contrastive learning (no labels)
2. **Fine-tune**: Adapt to downstream task with small labeled dataset
3. **Deploy**: Use learned features for inference

**Advantages**:
- Reduces labeled data requirements
- Better generalization
- Transfer learning capabilities

### Implementation

In [ ]:
class SimpleEncoder(nn.Module):
    """
    Simple CNN encoder for contrastive learning
    """
    def __init__(self, embedding_dim=128):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten()
        )
        self.projection_head = nn.Sequential(
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, embedding_dim)
        )
    
    def forward(self, x):
        features = self.encoder(x)
        embeddings = self.projection_head(features)
        return embeddings


# Initialize model
model = SimpleEncoder(embedding_dim=128)
print(f"✅ Model initialized with {sum(p.numel() for p in model.parameters()):,} parameters")

# Create synthetic training data (for demonstration)
# In practice, use real datasets like CIFAR-10
def create_synthetic_batch(batch_size=32, img_size=32):
    """Create synthetic image pairs for demonstration"""
    # Generate random images
    img1 = torch.randn(batch_size, 3, img_size, img_size)
    # Create augmented version (add small noise)
    img2 = img1 + torch.randn_like(img1) * 0.1
    return img1, img2

# Training loop (simplified)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 5
losses = []

print("\n🏋️ Training contrastive encoder...")
model.train()

for epoch in range(num_epochs):
    epoch_losses = []
    
    # Simulate 10 batches per epoch
    for batch_idx in range(10):
        # Get batch
        img1, img2 = create_synthetic_batch(batch_size=32)
        
        # Forward pass
        z1 = model(img1)
        z2 = model(img2)
        
        # Compute loss
        loss = contrastive_loss(z1, z2, temperature=0.5)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_losses.append(loss.item())
    
    avg_loss = np.mean(epoch_losses)
    losses.append(avg_loss)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f}")

print("\n✅ Training completed!")

### 📊 Visualization: Training Progress

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Training loss
axes[0].plot(range(1, num_epochs+1), losses, 'o-', linewidth=2, markersize=8)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Contrastive Loss', fontsize=12)
axes[0].set_title('Training Loss Curve', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Plot 2: Embedding space visualization (t-SNE)
model.eval()
with torch.no_grad():
    # Generate embeddings for multiple images
    embeddings_list = []
    labels_list = []
    
    for i in range(5):  # 5 different "classes"
        img_batch = torch.randn(20, 3, 32, 32) + i * 0.5  # Slight shift per class
        embeds = model(img_batch)
        embeddings_list.append(embeds.numpy())
        labels_list.extend([i] * 20)
    
    embeddings = np.vstack(embeddings_list)
    labels = np.array(labels_list)

# Apply t-SNE
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=42)
embeddings_2d = tsne.fit_transform(embeddings)

# Plot
scatter = axes[1].scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], 
                          c=labels, cmap='tab10', s=100, alpha=0.7, edgecolors='black')
axes[1].set_xlabel('t-SNE Dimension 1', fontsize=12)
axes[1].set_ylabel('t-SNE Dimension 2', fontsize=12)
axes[1].set_title('Learned Embedding Space (t-SNE)', fontsize=14, fontweight='bold')
plt.colorbar(scatter, ax=axes[1], label='Class')

plt.tight_layout()
plt.show()

print("\n📊 Key Insights:")
print("1. Loss decreases → Model learning useful representations")
print("2. Similar samples cluster together in embedding space")
print("3. Self-supervised learning discovers structure without labels")

### 🎯 Your Turn: Practice Task

**Task**: Extend the training pipeline to:
1. Compare self-supervised vs supervised learning with limited labels
2. Add a linear classifier on top of frozen features
3. Evaluate on a downstream classification task
4. Plot: Accuracy vs Number of Labeled Samples

**Expected Result**: Self-supervised pretraining should outperform training from scratch with few labels.

---

# 📈 Part 3: Time Series Clustering

Time series data has unique characteristics:
- **Variable lengths**: Different durations
- **Phase shifts**: Time warping between patterns
- **Noise**: Random fluctuations
- **Sampling rates**: Inconsistent frequencies

**Challenge**: Euclidean distance is inadequate for comparing time series!

## Key Methods:
- **DTW (Dynamic Time Warping)**: Optimal alignment of sequences
- **K-Shape**: Shape-based clustering with cross-correlation
- **Matrix Profile**: Subsequence pattern discovery

---

## 📝 Exercise 4: Understanding Time Series Characteristics

### Concept

Time series data exhibits several important properties:
- **Trend**: Long-term increase/decrease
- **Seasonality**: Periodic patterns
- **Noise**: Random variations
- **Phase shifts**: Temporal misalignment

### Implementation

In [ ]:
def generate_time_series(n_points=100, series_type='trend'):
    """
    Generate synthetic time series with different characteristics
    """
    t = np.linspace(0, 10, n_points)
    
    if series_type == 'trend':
        # Linear trend with noise
        series = 0.5 * t + np.random.normal(0, 0.5, n_points)
        
    elif series_type == 'seasonal':
        # Seasonal pattern
        series = np.sin(2 * np.pi * t) + np.random.normal(0, 0.1, n_points)
        
    elif series_type == 'trend_seasonal':
        # Trend + Seasonality
        series = 0.3 * t + np.sin(2 * np.pi * t) + np.random.normal(0, 0.2, n_points)
        
    elif series_type == 'noisy':
        # High noise
        series = np.sin(2 * np.pi * t) + np.random.normal(0, 1.0, n_points)
        
    elif series_type == 'phase_shift':
        # Phase-shifted sine wave
        phase = np.random.uniform(0, 2 * np.pi)
        series = np.sin(2 * np.pi * t + phase) + np.random.normal(0, 0.1, n_points)
    
    return t, series


# Generate different types of time series
series_types = ['trend', 'seasonal', 'trend_seasonal', 'noisy', 'phase_shift']
time_series_data = {}

for ts_type in series_types:
    t, series = generate_time_series(n_points=150, series_type=ts_type)
    time_series_data[ts_type] = (t, series)

print("✅ Generated 5 different types of time series")

### 📊 Visualization: Time Series Types

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(14, 10))
axes = axes.flatten()

titles = {
    'trend': 'Trend Pattern',
    'seasonal': 'Seasonal Pattern',
    'trend_seasonal': 'Trend + Seasonal',
    'noisy': 'Noisy Signal',
    'phase_shift': 'Phase-Shifted Pattern'
}

for idx, (ts_type, (t, series)) in enumerate(time_series_data.items()):
    axes[idx].plot(t, series, linewidth=2, color=f'C{idx}')
    axes[idx].set_title(titles[ts_type], fontsize=13, fontweight='bold')
    axes[idx].set_xlabel('Time', fontsize=11)
    axes[idx].set_ylabel('Value', fontsize=11)
    axes[idx].grid(True, alpha=0.3)

# Add comparison plot
axes[5].plot(time_series_data['seasonal'][0], time_series_data['seasonal'][1], 
             label='Original', linewidth=2, color='blue')
axes[5].plot(time_series_data['phase_shift'][0], time_series_data['phase_shift'][1], 
             label='Phase-Shifted', linewidth=2, color='red', linestyle='--')
axes[5].set_title('Comparison: Phase Shift Challenge', fontsize=13, fontweight='bold')
axes[5].set_xlabel('Time', fontsize=11)
axes[5].set_ylabel('Value', fontsize=11)
axes[5].legend(fontsize=10)
axes[5].grid(True, alpha=0.3)

plt.suptitle('Time Series Data Characteristics', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("\n📊 Key Insights:")
print("1. Different patterns require different distance metrics")
print("2. Phase shifts are problematic for Euclidean distance")
print("3. Noise can obscure underlying patterns")

### 🎯 Your Turn: Practice Task

**Task**: Generate and analyze your own time series:
1. Create a time series with multiple components (trend + seasonal + noise)
2. Decompose it using seasonal decomposition
3. Compare Euclidean vs DTW distance on shifted versions

**Bonus**: Implement autocorrelation analysis to detect periodicity.

---

## 📝 Exercise 5: Dynamic Time Warping (DTW)

### Concept

**Dynamic Time Warping** finds the optimal alignment between two time series by:
- Allowing one-to-many matching
- Handling speed variations
- Computing minimum distance path through cost matrix

**DTW Algorithm**:
1. Build cost matrix: $D(i,j) = |x_i - y_j| + \min\{D(i-1,j), D(i,j-1), D(i-1,j-1)\}$
2. Find optimal warping path
3. Sum distances along path

**Complexity**: O(n²) where n is series length

### Implementation

In [ ]:
def dtw_distance(series1, series2):
    """
    Compute Dynamic Time Warping distance between two time series
    
    Args:
        series1: First time series (array)
        series2: Second time series (array)
    
    Returns:
        distance: DTW distance
        cost_matrix: Cost matrix for visualization
        path: Optimal warping path
    """
    n, m = len(series1), len(series2)
    
    # Initialize cost matrix with infinity
    cost_matrix = np.full((n+1, m+1), np.inf)
    cost_matrix[0, 0] = 0
    
    # Fill cost matrix
    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = abs(series1[i-1] - series2[j-1])
            cost_matrix[i, j] = cost + min(
                cost_matrix[i-1, j],    # insertion
                cost_matrix[i, j-1],    # deletion
                cost_matrix[i-1, j-1]   # match
            )
    
    # Backtrack to find optimal path
    path = []
    i, j = n, m
    while i > 0 and j > 0:
        path.append((i-1, j-1))
        
        # Choose minimum predecessor
        candidates = [
            (cost_matrix[i-1, j], (i-1, j)),
            (cost_matrix[i, j-1], (i, j-1)),
            (cost_matrix[i-1, j-1], (i-1, j-1))
        ]
        _, (i, j) = min(candidates, key=lambda x: x[0])
    
    path.reverse()
    distance = cost_matrix[n, m]
    
    return distance, cost_matrix[1:, 1:], path


# Generate two similar time series with phase shift
t = np.linspace(0, 4*np.pi, 100)
series1 = np.sin(t) + np.random.normal(0, 0.1, 100)
series2 = np.sin(t + np.pi/4) + np.random.normal(0, 0.1, 100)  # Phase shifted

# Compute DTW distance
dtw_dist, cost_matrix, dtw_path = dtw_distance(series1, series2)

# Compute Euclidean distance for comparison
euclidean_dist = np.sqrt(np.sum((series1 - series2)**2))

print(f"✅ DTW Distance: {dtw_dist:.4f}")
print(f"📏 Euclidean Distance: {euclidean_dist:.4f}")
print(f"🛤️  Warping path length: {len(dtw_path)}")

### 📊 Visualization: DTW Alignment

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Original time series
axes[0, 0].plot(series1, label='Series 1', linewidth=2, color='blue')
axes[0, 0].plot(series2, label='Series 2 (phase-shifted)', linewidth=2, color='red', linestyle='--')
axes[0, 0].set_title('Original Time Series', fontsize=13, fontweight='bold')
axes[0, 0].set_xlabel('Time Index', fontsize=11)
axes[0, 0].set_ylabel('Value', fontsize=11)
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: DTW cost matrix with path
im = axes[0, 1].imshow(cost_matrix, cmap='YlOrRd', aspect='auto', origin='lower')
plt.colorbar(im, ax=axes[0, 1], label='Cumulative Cost')

# Plot optimal path
path_i = [p[0] for p in dtw_path]
path_j = [p[1] for p in dtw_path]
axes[0, 1].plot(path_j, path_i, 'blue', linewidth=2, label='Optimal Path')
axes[0, 1].set_title('DTW Cost Matrix & Warping Path', fontsize=13, fontweight='bold')
axes[0, 1].set_xlabel('Series 2 Index', fontsize=11)
axes[0, 1].set_ylabel('Series 1 Index', fontsize=11)
axes[0, 1].legend(fontsize=10)

# Plot 3: Aligned series with connections
# Sample every 5th point to avoid clutter
sample_indices = range(0, len(dtw_path), 5)
for idx in sample_indices:
    i, j = dtw_path[idx]
    axes[1, 0].plot([i, j], [series1[i], series2[j]], 'gray', alpha=0.3, linewidth=1)

axes[1, 0].plot(series1, label='Series 1', linewidth=2, color='blue')
axes[1, 0].plot(series2, label='Series 2', linewidth=2, color='red', linestyle='--')
axes[1, 0].set_title('DTW Alignment (sampled connections)', fontsize=13, fontweight='bold')
axes[1, 0].set_xlabel('Time Index', fontsize=11)
axes[1, 0].set_ylabel('Value', fontsize=11)
axes[1, 0].legend(fontsize=10)
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Distance comparison
distances = ['DTW', 'Euclidean']
values = [dtw_dist, euclidean_dist]
colors = ['steelblue', 'coral']

bars = axes[1, 1].bar(distances, values, color=colors, edgecolor='black', linewidth=2)
axes[1, 1].set_title('Distance Metric Comparison', fontsize=13, fontweight='bold')
axes[1, 1].set_ylabel('Distance Value', fontsize=11)
axes[1, 1].grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, val in zip(bars, values):
    height = bar.get_height()
    axes[1, 1].text(bar.get_x() + bar.get_width()/2., height,
                    f'{val:.2f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.suptitle('Dynamic Time Warping Analysis', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print("\n📊 Key Insights:")
print("1. DTW finds optimal alignment despite phase shift")
print("2. Warping path shows how points are matched")
print("3. DTW is more robust to temporal distortions")

### 🎯 Your Turn: Practice Task

**Task**: Implement and test DTW with constraints:
1. Add Sakoe-Chiba band (window constraint)
2. Compare constrained vs unconstrained DTW
3. Measure computation time vs accuracy trade-off
4. Test on real data (e.g., speech or ECG signals)

**Bonus**: Implement FastDTW for improved efficiency.

---

## 📝 Exercise 6: DTW-based Clustering

### Concept

**DTW-based Clustering** combines DTW distance with clustering algorithms:

**Methods**:
1. **K-medoids**: Uses pairwise DTW distances (better than K-means)
2. **Hierarchical**: Creates dendrogram with DTW linkage
3. **DBA (DTW Barycenter Averaging)**: Computes cluster centroids

**Applications**:
- ECG pattern analysis
- Gesture recognition
- Stock price clustering
- Speech recognition

### Implementation

In [ ]:
def create_time_series_dataset(n_series=30, n_points=100, n_clusters=3):
    """
    Create synthetic time series dataset with different patterns
    """
    data = []
    labels = []
    
    t = np.linspace(0, 10, n_points)
    
    for cluster_id in range(n_clusters):
        for i in range(n_series // n_clusters):
            # Different patterns for each cluster
            if cluster_id == 0:
                # Low frequency sine wave
                series = np.sin(t) + np.random.normal(0, 0.2, n_points)
            elif cluster_id == 1:
                # High frequency sine wave
                series = np.sin(3 * t) + np.random.normal(0, 0.2, n_points)
            else:
                # Linear trend
                series = 0.3 * t + np.sin(t) + np.random.normal(0, 0.2, n_points)
            
            data.append(series)
            labels.append(cluster_id)
    
    return np.array(data), np.array(labels)


# Generate dataset
X_ts, y_true = create_time_series_dataset(n_series=30, n_points=100, n_clusters=3)
print(f"✅ Created dataset: {X_ts.shape[0]} time series, {X_ts.shape[1]} points each")

# Compute DTW distance matrix
def compute_dtw_distance_matrix(time_series_list):
    """
    Compute pairwise DTW distance matrix
    """
    n = len(time_series_list)
    distance_matrix = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i+1, n):
            dist, _, _ = dtw_distance(time_series_list[i], time_series_list[j])
            distance_matrix[i, j] = dist
            distance_matrix[j, i] = dist
    
    return distance_matrix

print("\n⏳ Computing DTW distance matrix (this may take a moment)...")
dtw_dist_matrix = compute_dtw_distance_matrix(X_ts)
print(f"✅ Distance matrix computed: {dtw_dist_matrix.shape}")

# Hierarchical clustering with DTW
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram

# Convert distance matrix to condensed form for scipy
from scipy.spatial.distance import squareform
condensed_dist = squareform(dtw_dist_matrix)

# Perform hierarchical clustering
linkage_matrix = linkage(condensed_dist, method='average')
y_pred = fcluster(linkage_matrix, t=3, criterion='maxclust')

# Adjust labels to start from 0
y_pred = y_pred - 1

# Evaluate clustering
ari = adjusted_rand_score(y_true, y_pred)
print(f"\n📊 Adjusted Rand Index: {ari:.4f}")
print(f"✅ Clustering completed!")

### 📊 Visualization: DTW Clustering Results

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Dendrogram
dendrogram(linkage_matrix, ax=axes[0, 0], color_threshold=50)
axes[0, 0].set_title('Hierarchical Clustering Dendrogram', fontsize=13, fontweight='bold')
axes[0, 0].set_xlabel('Sample Index', fontsize=11)
axes[0, 0].set_ylabel('DTW Distance', fontsize=11)
axes[0, 0].grid(True, alpha=0.3, axis='y')

# Plot 2: Distance matrix heatmap
im = axes[0, 1].imshow(dtw_dist_matrix, cmap='viridis', aspect='auto')
plt.colorbar(im, ax=axes[0, 1], label='DTW Distance')
axes[0, 1].set_title('DTW Distance Matrix', fontsize=13, fontweight='bold')
axes[0, 1].set_xlabel('Series Index', fontsize=11)
axes[0, 1].set_ylabel('Series Index', fontsize=11)

# Plot 3: True clusters
for cluster_id in range(3):
    cluster_series = X_ts[y_true == cluster_id]
    for series in cluster_series:
        axes[1, 0].plot(series, alpha=0.5, linewidth=1, color=f'C{cluster_id}')
    
    # Plot cluster centroid
    centroid = cluster_series.mean(axis=0)
    axes[1, 0].plot(centroid, linewidth=3, color=f'C{cluster_id}', 
                    label=f'Cluster {cluster_id}', linestyle='--')

axes[1, 0].set_title('True Clusters', fontsize=13, fontweight='bold')
axes[1, 0].set_xlabel('Time Index', fontsize=11)
axes[1, 0].set_ylabel('Value', fontsize=11)
axes[1, 0].legend(fontsize=10)
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Predicted clusters
for cluster_id in range(3):
    cluster_series = X_ts[y_pred == cluster_id]
    for series in cluster_series:
        axes[1, 1].plot(series, alpha=0.5, linewidth=1, color=f'C{cluster_id}')
    
    # Plot cluster centroid
    if len(cluster_series) > 0:
        centroid = cluster_series.mean(axis=0)
        axes[1, 1].plot(centroid, linewidth=3, color=f'C{cluster_id}', 
                        label=f'Cluster {cluster_id}', linestyle='--')

axes[1, 1].set_title(f'DTW Clustering (ARI={ari:.3f})', fontsize=13, fontweight='bold')
axes[1, 1].set_xlabel('Time Index', fontsize=11)
axes[1, 1].set_ylabel('Value', fontsize=11)
axes[1, 1].legend(fontsize=10)
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('DTW-based Time Series Clustering', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print("\n📊 Key Insights:")
print("1. DTW effectively groups similar temporal patterns")
print("2. Dendrogram shows hierarchical structure")
print("3. Distance matrix reveals cluster boundaries")

---

# 🕸️ Part 4: Graph Clustering

Graph clustering identifies communities or groups of densely connected nodes.

## Key Concepts:
- **Spectral Clustering**: Uses graph Laplacian eigendecomposition
- **Louvain Algorithm**: Optimizes modularity for community detection
- **GNN (Graph Neural Networks)**: Deep learning on graphs
- **Graph Convolutional Networks**: Learn node embeddings

---

## 📝 Exercise 7: Introduction to Graph Data

### Concept

A **graph** G = (V, E) consists of:
- **Nodes (V)**: Entities (people, proteins, web pages)
- **Edges (E)**: Relationships or connections
- **Adjacency Matrix (A)**: A[i,j] = 1 if edge exists, 0 otherwise

**Graph Properties**:
- Degree: Number of connections per node
- Clustering coefficient: Local connectivity
- Connected components: Separated subgraphs

### Implementation

In [ ]:
# Create sample graphs
def create_sample_graphs():
    """Create various graph structures"""
    
    # 1. Karate Club - famous social network
    G_karate = nx.karate_club_graph()
    
    # 2. Small world network
    G_small_world = nx.watts_strogatz_graph(n=50, k=4, p=0.1, seed=42)
    
    # 3. Scale-free network (preferential attachment)
    G_scale_free = nx.barabasi_albert_graph(n=50, m=2, seed=42)
    
    # 4. Community structure
    sizes = [15, 15, 15]
    probs = [[0.7, 0.1, 0.1],
             [0.1, 0.7, 0.1],
             [0.1, 0.1, 0.7]]
    G_communities = nx.stochastic_block_model(sizes, probs, seed=42)
    
    return G_karate, G_small_world, G_scale_free, G_communities


# Create graphs
G_karate, G_small_world, G_scale_free, G_communities = create_sample_graphs()

# Compute graph statistics
graphs = {
    'Karate Club': G_karate,
    'Small World': G_small_world,
    'Scale-Free': G_scale_free,
    'Communities': G_communities
}

print("📊 Graph Statistics:\n")
for name, G in graphs.items():
    print(f"{name}:")
    print(f"  Nodes: {G.number_of_nodes()}")
    print(f"  Edges: {G.number_of_edges()}")
    print(f"  Avg Degree: {sum(dict(G.degree()).values()) / G.number_of_nodes():.2f}")
    print(f"  Clustering Coef: {nx.average_clustering(G):.3f}")
    print()

### 📊 Visualization: Graph Structures

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

graph_list = [
    (G_karate, 'Karate Club Network'),
    (G_small_world, 'Small World Network'),
    (G_scale_free, 'Scale-Free Network'),
    (G_communities, 'Community Structure')
]

for idx, (G, title) in enumerate(graph_list):
    # Compute layout
    if idx == 0:  # Karate club
        pos = nx.spring_layout(G, seed=42, k=0.5)
    else:
        pos = nx.spring_layout(G, seed=42)
    
    # Draw network
    nx.draw_networkx_nodes(G, pos, 
                          node_color=range(G.number_of_nodes()),
                          cmap='viridis',
                          node_size=300,
                          ax=axes[idx])
    nx.draw_networkx_edges(G, pos, alpha=0.3, ax=axes[idx])
    
    axes[idx].set_title(title, fontsize=14, fontweight='bold')
    axes[idx].axis('off')

plt.suptitle('Different Graph Structures', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print("\n📊 Key Insights:")
print("1. Different networks have distinct structural properties")
print("2. Community structure shows clear clusters")
print("3. Scale-free networks have hub nodes")

---

## 📝 Exercise 8: Spectral Clustering on Graphs

### Concept

**Spectral Clustering** uses the spectrum (eigenvalues) of the graph Laplacian:

**Algorithm**:
1. Compute Graph Laplacian: L = D - A
   - D: Degree matrix (diagonal)
   - A: Adjacency matrix
2. Compute k smallest eigenvectors
3. Use eigenvectors as features
4. Apply K-means clustering

**Advantages**:
- Handles non-convex clusters
- Based on graph cut objectives
- Well-suited for community detection

### Implementation

In [ ]:
def spectral_clustering_analysis(G, n_clusters=3):
    """
    Perform spectral clustering on graph
    """
    # Get adjacency matrix
    A = nx.to_numpy_array(G)
    
    # Compute degree matrix
    degrees = A.sum(axis=1)
    D = np.diag(degrees)
    
    # Compute Laplacian
    L = D - A
    
    # Compute normalized Laplacian
    D_inv_sqrt = np.diag(1.0 / np.sqrt(degrees + 1e-10))
    L_norm = np.eye(len(G)) - D_inv_sqrt @ A @ D_inv_sqrt
    
    # Eigendecomposition
    eigenvalues, eigenvectors = np.linalg.eigh(L_norm)
    
    # Sort by eigenvalue
    idx = eigenvalues.argsort()
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    
    # Use k smallest eigenvectors (skip first - constant)
    embedding = eigenvectors[:, 1:n_clusters+1]
    
    # K-means on embedding
    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    labels = kmeans.fit_predict(embedding)
    
    return labels, eigenvalues, embedding


# Apply spectral clustering to community graph
labels_spectral, eigenvalues, embedding = spectral_clustering_analysis(G_communities, n_clusters=3)

print("✅ Spectral clustering completed!")
print(f"\n📊 Cluster sizes: {np.bincount(labels_spectral)}")
print(f"\n🔢 First 10 eigenvalues: {eigenvalues[:10].round(3)}")

### 📊 Visualization: Spectral Clustering

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Eigenvalue spectrum
axes[0, 0].plot(eigenvalues[:20], 'o-', linewidth=2, markersize=8)
axes[0, 0].axvline(x=3, color='red', linestyle='--', label='Number of clusters', linewidth=2)
axes[0, 0].set_xlabel('Index', fontsize=12)
axes[0, 0].set_ylabel('Eigenvalue', fontsize=12)
axes[0, 0].set_title('Graph Laplacian Spectrum', fontsize=13, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Spectral embedding (2D)
if embedding.shape[1] >= 2:
    scatter = axes[0, 1].scatter(embedding[:, 0], embedding[:, 1], 
                                 c=labels_spectral, cmap='viridis', 
                                 s=100, alpha=0.7, edgecolors='black')
    axes[0, 1].set_xlabel('1st Eigenvector', fontsize=12)
    axes[0, 1].set_ylabel('2nd Eigenvector', fontsize=12)
    axes[0, 1].set_title('Spectral Embedding Space', fontsize=13, fontweight='bold')
    plt.colorbar(scatter, ax=axes[0, 1], label='Cluster')
    axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Original graph with true communities
pos = nx.spring_layout(G_communities, seed=42)
true_labels = [G_communities.nodes[i]['block'] for i in G_communities.nodes()]
nx.draw_networkx_nodes(G_communities, pos, node_color=true_labels, 
                      cmap='viridis', node_size=200, ax=axes[1, 0])
nx.draw_networkx_edges(G_communities, pos, alpha=0.2, ax=axes[1, 0])
axes[1, 0].set_title('True Communities', fontsize=13, fontweight='bold')
axes[1, 0].axis('off')

# Plot 4: Detected clusters
nx.draw_networkx_nodes(G_communities, pos, node_color=labels_spectral, 
                      cmap='viridis', node_size=200, ax=axes[1, 1])
nx.draw_networkx_edges(G_communities, pos, alpha=0.2, ax=axes[1, 1])

# Compute accuracy
ari = adjusted_rand_score(true_labels, labels_spectral)
axes[1, 1].set_title(f'Spectral Clustering (ARI={ari:.3f})', fontsize=13, fontweight='bold')
axes[1, 1].axis('off')

plt.suptitle('Spectral Clustering Analysis', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print("\n📊 Key Insights:")
print("1. Eigenvalue gap indicates number of clusters")
print("2. Spectral embedding separates communities")
print("3. Successfully detects graph structure")

---

## 📝 Exercise 9: Louvain Community Detection

### Concept

**Louvain Algorithm** optimizes modularity Q:

$$Q = \frac{1}{2m} \sum_{ij} \left[A_{ij} - \frac{k_i k_j}{2m}\right] \delta(c_i, c_j)$$

Where:
- $A_{ij}$: Adjacency matrix
- $k_i$: Degree of node i
- $m$: Total edges
- $c_i$: Community of node i

**Two-Phase Approach**:
1. **Local optimization**: Move nodes to maximize modularity
2. **Network aggregation**: Build new network from communities

**Advantages**: Fast, scalable, hierarchical structure

### Implementation

In [ ]:
# Simple Louvain implementation
# Note: In practice, use python-louvain library

try:
    import community as community_louvain
    print("✅ Using python-louvain library")
    has_louvain = True
except ImportError:
    print("⚠️  python-louvain not installed, using greedy modularity")
    has_louvain = False

def detect_communities(G):
    """Detect communities using available method"""
    if has_louvain:
        # Use Louvain algorithm
        partition = community_louvain.best_partition(G)
        communities = list(partition.values())
    else:
        # Use greedy modularity communities
        from networkx.algorithms import community as nx_community
        communities_gen = nx_community.greedy_modularity_communities(G)
        
        # Convert to node labels
        node_to_community = {}
        for idx, comm in enumerate(communities_gen):
            for node in comm:
                node_to_community[node] = idx
        
        communities = [node_to_community[i] for i in range(G.number_of_nodes())]
    
    return np.array(communities)


# Apply to Karate Club (famous example)
communities_karate = detect_communities(G_karate)

# Compute modularity
def compute_modularity(G, communities):
    """Compute modularity score"""
    from networkx.algorithms import community as nx_community
    
    # Convert to community sets
    unique_comms = set(communities)
    community_sets = [set(np.where(communities == c)[0]) for c in unique_comms]
    
    modularity = nx_community.modularity(G, community_sets)
    return modularity

modularity = compute_modularity(G_karate, communities_karate)

print(f"\n✅ Community detection completed!")
print(f"📊 Number of communities: {len(set(communities_karate))}")
print(f"📊 Modularity Q: {modularity:.4f}")
print(f"📊 Community sizes: {np.bincount(communities_karate)}")

### 📊 Visualization: Louvain Communities

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Layout
pos = nx.spring_layout(G_karate, seed=42, k=0.5)

# Plot 1: Ground truth (known factions)
true_clubs = [G_karate.nodes[i]['club'] for i in G_karate.nodes()]
true_labels = [0 if club == 'Mr. Hi' else 1 for club in true_clubs]

nx.draw_networkx_nodes(G_karate, pos, node_color=true_labels,
                      cmap='coolwarm', node_size=500,
                      edgecolors='black', linewidths=2,
                      ax=axes[0])
nx.draw_networkx_edges(G_karate, pos, alpha=0.3, ax=axes[0])
nx.draw_networkx_labels(G_karate, pos, font_size=8, ax=axes[0])
axes[0].set_title('Karate Club: True Factions', fontsize=14, fontweight='bold')
axes[0].axis('off')

# Plot 2: Detected communities
nx.draw_networkx_nodes(G_karate, pos, node_color=communities_karate,
                      cmap='viridis', node_size=500,
                      edgecolors='black', linewidths=2,
                      ax=axes[1])
nx.draw_networkx_edges(G_karate, pos, alpha=0.3, ax=axes[1])
nx.draw_networkx_labels(G_karate, pos, font_size=8, ax=axes[1])

ari_karate = adjusted_rand_score(true_labels, communities_karate)
axes[1].set_title(f'Detected Communities\n(Modularity={modularity:.3f}, ARI={ari_karate:.3f})',
                 fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.suptitle('Louvain Community Detection', fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

print("\n📊 Key Insights:")
print("1. Louvain successfully detects community structure")
print("2. High modularity indicates strong communities")
print("3. Matches known factions in Karate Club")

---

# 🚀 Part 5: Advanced Topics

Modern clustering approaches for complex scenarios:

## Topics:
- **Multi-modal Clustering**: Combining different data types
- **Large-Scale Clustering**: Billions of samples
- **Deep Clustering**: End-to-end learned representations

---

## 📝 Exercise 10: Multi-modal Clustering

### Concept

**Multi-modal clustering** handles data with multiple modalities (types):

**Approaches**:
1. **Early Fusion**: Concatenate features from all modalities
2. **Late Fusion**: Cluster each modality separately, then combine
3. **Deep Multi-modal**: Learn shared representation space

**Challenge**: Different feature spaces, scales, and semantics

**Applications**:
- Video (visual + audio)
- Medical (images + reports)
- E-commerce (product images + descriptions)

### Implementation

In [ ]:
def create_multimodal_dataset(n_samples=200, n_clusters=4):
    """
    Create synthetic multi-modal dataset
    
    Returns:
        visual_features: Image-like features (2D)
        text_features: Text-like features (2D)
        true_labels: Ground truth clusters
    """
    # Generate cluster centers
    np.random.seed(42)
    
    # Visual modality (e.g., color, shape)
    visual_centers = np.random.randn(n_clusters, 2) * 3
    visual_features = []
    
    # Text modality (e.g., semantic embeddings)
    text_centers = np.random.randn(n_clusters, 2) * 3
    text_features = []
    
    true_labels = []
    
    for cluster_id in range(n_clusters):
        n_per_cluster = n_samples // n_clusters
        
        # Visual features
        visual = visual_centers[cluster_id] + np.random.randn(n_per_cluster, 2) * 0.5
        visual_features.append(visual)
        
        # Text features (slightly different distribution)
        text = text_centers[cluster_id] + np.random.randn(n_per_cluster, 2) * 0.7
        text_features.append(text)
        
        true_labels.extend([cluster_id] * n_per_cluster)
    
    visual_features = np.vstack(visual_features)
    text_features = np.vstack(text_features)
    true_labels = np.array(true_labels)
    
    return visual_features, text_features, true_labels


# Create dataset
visual_features, text_features, true_labels = create_multimodal_dataset()

# Normalize features
from sklearn.preprocessing import StandardScaler
scaler_v = StandardScaler()
scaler_t = StandardScaler()

visual_norm = scaler_v.fit_transform(visual_features)
text_norm = scaler_t.fit_transform(text_features)

print(f"✅ Created multi-modal dataset:")
print(f"   Visual features: {visual_features.shape}")
print(f"   Text features: {text_features.shape}")
print(f"   True clusters: {len(set(true_labels))}")

In [ ]:
# Three fusion approaches

# 1. Early Fusion: Concatenate features
early_fusion_features = np.hstack([visual_norm, text_norm])
kmeans_early = KMeans(n_clusters=4, random_state=42, n_init=10)
labels_early = kmeans_early.fit_predict(early_fusion_features)
ari_early = adjusted_rand_score(true_labels, labels_early)

# 2. Late Fusion: Cluster separately, then combine
kmeans_visual = KMeans(n_clusters=4, random_state=42, n_init=10)
kmeans_text = KMeans(n_clusters=4, random_state=42, n_init=10)

labels_visual = kmeans_visual.fit_predict(visual_norm)
labels_text = kmeans_text.fit_predict(text_norm)

# Combine by voting (simple approach)
labels_late = []
for v, t in zip(labels_visual, labels_text):
    labels_late.append(v if np.random.rand() > 0.5 else t)
labels_late = np.array(labels_late)
ari_late = adjusted_rand_score(true_labels, labels_late)

# 3. Weighted Fusion: Weight modalities by reliability
# Use silhouette scores as weights
from sklearn.metrics import silhouette_score
sil_visual = silhouette_score(visual_norm, labels_visual)
sil_text = silhouette_score(text_norm, labels_text)

# Normalize weights
weight_visual = sil_visual / (sil_visual + sil_text)
weight_text = sil_text / (sil_visual + sil_text)

# Weighted feature combination
weighted_features = np.hstack([
    visual_norm * weight_visual,
    text_norm * weight_text
])
kmeans_weighted = KMeans(n_clusters=4, random_state=42, n_init=10)
labels_weighted = kmeans_weighted.fit_predict(weighted_features)
ari_weighted = adjusted_rand_score(true_labels, labels_weighted)

print("\n📊 Fusion Strategy Comparison:")
print(f"\nEarly Fusion (concatenate):")
print(f"  ARI: {ari_early:.4f}")
print(f"\nLate Fusion (combine votes):")
print(f"  ARI: {ari_late:.4f}")
print(f"\nWeighted Fusion:")
print(f"  Visual weight: {weight_visual:.3f}")
print(f"  Text weight: {weight_text:.3f}")
print(f"  ARI: {ari_weighted:.4f}")

### 📊 Visualization: Multi-modal Clustering

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# Row 1: Individual modalities
# Visual features - True labels
scatter1 = axes[0, 0].scatter(visual_features[:, 0], visual_features[:, 1],
                              c=true_labels, cmap='viridis', s=50, alpha=0.7, edgecolors='black')
axes[0, 0].set_title('Visual Features (Ground Truth)', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Visual Dim 1')
axes[0, 0].set_ylabel('Visual Dim 2')
axes[0, 0].grid(True, alpha=0.3)

# Text features - True labels
scatter2 = axes[0, 1].scatter(text_features[:, 0], text_features[:, 1],
                              c=true_labels, cmap='viridis', s=50, alpha=0.7, edgecolors='black')
axes[0, 1].set_title('Text Features (Ground Truth)', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Text Dim 1')
axes[0, 1].set_ylabel('Text Dim 2')
axes[0, 1].grid(True, alpha=0.3)

# Combined view
axes[0, 2].scatter(visual_features[:, 0], visual_features[:, 1],
                  c=true_labels, cmap='viridis', s=50, alpha=0.5, 
                  marker='o', label='Visual', edgecolors='black')
axes[0, 2].scatter(text_features[:, 0], text_features[:, 1],
                  c=true_labels, cmap='viridis', s=50, alpha=0.5, 
                  marker='^', label='Text', edgecolors='black')
axes[0, 2].set_title('Both Modalities', fontsize=12, fontweight='bold')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

# Row 2: Fusion results
# Early fusion
axes[1, 0].scatter(visual_features[:, 0], visual_features[:, 1],
                  c=labels_early, cmap='viridis', s=50, alpha=0.7, edgecolors='black')
axes[1, 0].set_title(f'Early Fusion\n(ARI={ari_early:.3f})', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Visual Dim 1')
axes[1, 0].set_ylabel('Visual Dim 2')
axes[1, 0].grid(True, alpha=0.3)

# Late fusion
axes[1, 1].scatter(text_features[:, 0], text_features[:, 1],
                  c=labels_late, cmap='viridis', s=50, alpha=0.7, edgecolors='black')
axes[1, 1].set_title(f'Late Fusion\n(ARI={ari_late:.3f})', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Text Dim 1')
axes[1, 1].set_ylabel('Text Dim 2')
axes[1, 1].grid(True, alpha=0.3)

# Weighted fusion
axes[1, 2].scatter(visual_features[:, 0], visual_features[:, 1],
                  c=labels_weighted, cmap='viridis', s=50, alpha=0.7, edgecolors='black')
axes[1, 2].set_title(f'Weighted Fusion\n(ARI={ari_weighted:.3f})', 
                    fontsize=12, fontweight='bold')
axes[1, 2].set_xlabel('Visual Dim 1')
axes[1, 2].set_ylabel('Visual Dim 2')
axes[1, 2].grid(True, alpha=0.3)

plt.suptitle('Multi-modal Clustering Comparison', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print("\n📊 Key Insights:")
print("1. Different modalities capture complementary information")
print("2. Fusion strategy matters for performance")
print("3. Weighted fusion can improve results")

---

## 📝 Exercise 11: Large-Scale Clustering

### Concept

**Challenges** with large-scale data:
- Billions of samples
- High dimensionality
- Memory constraints
- Computation time

**Scalability Techniques**:
1. **Mini-batch K-means**: Process streaming data in batches
2. **Approximate NN**: Fast similarity search (FAISS)
3. **Sampling**: Use representative subset (CoreSets)
4. **GPU Acceleration**: Parallel computation

**Trade-off**: Speed ⚖️ Accuracy

### Implementation

In [ ]:
from sklearn.cluster import MiniBatchKMeans
import time

# Generate large dataset
print("⏳ Generating large-scale dataset...")
n_samples_large = 50000
n_features = 50
n_clusters = 10

X_large, y_large = make_blobs(
    n_samples=n_samples_large,
    n_features=n_features,
    centers=n_clusters,
    random_state=42
)

print(f"✅ Dataset created: {X_large.shape}")
print(f"   Size in memory: {X_large.nbytes / 1024**2:.2f} MB")

# Compare standard K-means vs Mini-batch K-means
print("\n🏃 Running clustering algorithms...\n")

# Standard K-means
print("1. Standard K-means:")
start = time.time()
kmeans_standard = KMeans(n_clusters=n_clusters, random_state=42, n_init=3, max_iter=100)
labels_standard = kmeans_standard.fit_predict(X_large)
time_standard = time.time() - start
inertia_standard = kmeans_standard.inertia_
ari_standard = adjusted_rand_score(y_large, labels_standard)
print(f"   Time: {time_standard:.2f}s")
print(f"   Inertia: {inertia_standard:.2f}")
print(f"   ARI: {ari_standard:.4f}")

# Mini-batch K-means
print("\n2. Mini-batch K-means:")
start = time.time()
kmeans_minibatch = MiniBatchKMeans(
    n_clusters=n_clusters, 
    random_state=42,
    batch_size=1000,
    max_iter=100,
    n_init=3
)
labels_minibatch = kmeans_minibatch.fit_predict(X_large)
time_minibatch = time.time() - start
inertia_minibatch = kmeans_minibatch.inertia_
ari_minibatch = adjusted_rand_score(y_large, labels_minibatch)
print(f"   Time: {time_minibatch:.2f}s")
print(f"   Inertia: {inertia_minibatch:.2f}")
print(f"   ARI: {ari_minibatch:.4f}")

# Sampling approach
print("\n3. Sampling + Standard K-means:")
sample_size = 5000
indices = np.random.choice(n_samples_large, sample_size, replace=False)
X_sample = X_large[indices]
y_sample = y_large[indices]

start = time.time()
kmeans_sample = KMeans(n_clusters=n_clusters, random_state=42, n_init=3)
kmeans_sample.fit(X_sample)
# Predict on full dataset using learned centroids
labels_sample = kmeans_sample.predict(X_large)
time_sample = time.time() - start
ari_sample = adjusted_rand_score(y_large, labels_sample)
print(f"   Time: {time_sample:.2f}s")
print(f"   Sample size: {sample_size} ({sample_size/n_samples_large*100:.1f}%)")
print(f"   ARI: {ari_sample:.4f}")

# Summary
print("\n" + "="*60)
print("📊 Performance Summary:")
print("="*60)
print(f"Speedup (Mini-batch vs Standard): {time_standard/time_minibatch:.2f}x")
print(f"Speedup (Sampling vs Standard): {time_standard/time_sample:.2f}x")
print(f"\nAccuracy (ARI) comparison:")
print(f"  Standard:   {ari_standard:.4f}")
print(f"  Mini-batch: {ari_minibatch:.4f} ({(ari_minibatch/ari_standard-1)*100:+.1f}%)")
print(f"  Sampling:   {ari_sample:.4f} ({(ari_sample/ari_standard-1)*100:+.1f}%)")

### 📊 Visualization: Scalability Analysis

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Plot 1: Time comparison
methods = ['Standard\nK-means', 'Mini-batch\nK-means', 'Sampling\n(10%)']
times = [time_standard, time_minibatch, time_sample]
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

bars = axes[0].bar(methods, times, color=colors, edgecolor='black', linewidth=2)
axes[0].set_ylabel('Time (seconds)', fontsize=12)
axes[0].set_title('Computation Time', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# Add value labels
for bar, time_val in zip(bars, times):
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height,
                f'{time_val:.2f}s', ha='center', va='bottom', fontweight='bold')

# Plot 2: ARI comparison
aris = [ari_standard, ari_minibatch, ari_sample]
bars = axes[1].bar(methods, aris, color=colors, edgecolor='black', linewidth=2)
axes[1].set_ylabel('ARI Score', fontsize=12)
axes[1].set_title('Clustering Quality (ARI)', fontsize=14, fontweight='bold')
axes[1].set_ylim([0, 1])
axes[1].grid(True, alpha=0.3, axis='y')

# Add value labels
for bar, ari in zip(bars, aris):
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2., height,
                f'{ari:.3f}', ha='center', va='bottom', fontweight='bold')

# Plot 3: Trade-off (Speed vs Accuracy)
speedups = [1.0, time_standard/time_minibatch, time_standard/time_sample]
ari_relative = [ari/ari_standard for ari in aris]

axes[2].scatter(speedups, ari_relative, s=500, c=colors, 
               edgecolors='black', linewidths=2, alpha=0.7)

# Add labels
for i, method in enumerate(['Standard', 'Mini-batch', 'Sampling']):
    axes[2].annotate(method, (speedups[i], ari_relative[i]),
                    xytext=(10, 10), textcoords='offset points',
                    fontsize=10, fontweight='bold',
                    bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.3))

axes[2].axhline(y=1.0, color='red', linestyle='--', linewidth=2, alpha=0.5, label='Baseline')
axes[2].set_xlabel('Speedup (x times faster)', fontsize=12)
axes[2].set_ylabel('Relative ARI (vs Standard)', fontsize=12)
axes[2].set_title('Speed vs Accuracy Trade-off', fontsize=14, fontweight='bold')
axes[2].legend(fontsize=10)
axes[2].grid(True, alpha=0.3)

plt.suptitle('Large-Scale Clustering: Performance Analysis', 
             fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("\n📊 Key Insights:")
print("1. Mini-batch K-means: Significant speedup with minimal accuracy loss")
print("2. Sampling: Fastest but may sacrifice some accuracy")
print("3. Choose method based on speed/accuracy requirements")

---

# 🎓 Summary & Key Takeaways

## What We Learned

### 🔄 Self-Supervised Learning
- **Contrastive learning** learns by comparing positive/negative pairs
- **SimCLR** uses strong augmentations to create views
- **InfoNCE loss** with temperature scaling
- Enables learning from unlabeled data at scale

### 📈 Time Series Clustering
- **DTW (Dynamic Time Warping)** handles temporal misalignment
- More robust than Euclidean distance for time series
- **K-Shape** offers faster alternative with shape-based similarity
- Applications: ECG, stocks, sensor data

### 🕸️ Graph Clustering
- **Spectral clustering** uses eigendecomposition
- **Louvain** optimizes modularity for communities
- Graph structure provides rich information
- Applications: Social networks, biological networks

### 🚀 Advanced Topics
- **Multi-modal**: Combine different data types
- **Large-scale**: Trade-off speed vs accuracy
- **Mini-batch** and **sampling** for scalability

---

## 🎯 Best Practices

1. **Choose appropriate distance metric**
   - Euclidean for standard data
   - DTW for time series
   - Graph distances for networks

2. **Preprocess data carefully**
   - Normalization/standardization
   - Handle missing values
   - Feature engineering

3. **Validate results**
   - Multiple metrics (Silhouette, ARI, modularity)
   - Visual inspection
   - Domain knowledge

4. **Consider scalability early**
   - Start with sampling for prototyping
   - Use mini-batch for large datasets
   - Leverage GPU when available

---

## 📚 Further Reading

### Papers
- **SimCLR**: Chen et al. (2020) - "A Simple Framework for Contrastive Learning"
- **DTW**: Sakoe & Chiba (1978) - "Dynamic Programming Algorithm Optimization"
- **Louvain**: Blondel et al. (2008) - "Fast Unfolding of Communities"
- **Spectral Clustering**: Ng et al. (2002) - "On Spectral Clustering"

### Libraries
- **Scikit-learn**: General clustering algorithms
- **TSlearn**: Time series clustering
- **NetworkX**: Graph algorithms
- **PyTorch Geometric**: Graph neural networks

---

## 🚀 Next Steps

1. **Apply to your own data**
   - Start with exploratory analysis
   - Try multiple methods
   - Compare results

2. **Experiment with parameters**
   - Number of clusters
   - Distance metrics
   - Preprocessing steps

3. **Combine approaches**
   - Ensemble clustering
   - Multi-view learning
   - Hierarchical methods

4. **Stay updated**
   - Latest research papers
   - New algorithms and tools
   - Community discussions

---

## 💡 Final Challenge

**Choose a real-world dataset and**:
1. Apply at least 3 different clustering methods
2. Compare their performance quantitatively
3. Visualize the results
4. Provide actionable insights

**Suggested datasets**:
- UCI Machine Learning Repository
- Kaggle datasets
- Your own domain data

---

### Thank you for completing this notebook! 🎉

**Questions or feedback?**
- Email: homin.park@ghent.ac.kr
- Based on Lecture 18: Advanced Unsupervised Learning

---